In [1]:
import os
from os.path import join
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import trading as trd

Using Theano backend.


In [3]:
model_folders = glob.glob(trd.MODEL_LOC + '\\*')

perf_all

In [4]:
model_names = []

perf_q = []
perf_rebal = []
perf_nothing = []

for m_dir in model_folders:
    mdl_name = os.path.basename(m_dir)
    model_names.append(mdl_name)

    perf_q.append(np.loadtxt(join(m_dir, 'q_perf.csv'))) 
    perf_rebal.append(np.loadtxt(join(m_dir, 'rebal_perf.csv'))) 
    perf_nothing.append(np.loadtxt(join(m_dir, 'nothing_perf.csv'))) 
    

perf_q = np.array(perf_q)
perf_rebal = np.array(perf_rebal)
perf_rebal = perf_rebal.mean(axis=0)
perf_nothing = np.array(perf_nothing)
perf_nothing = perf_nothing.mean(axis=0)

model_names.extend(["rebalance", "do nothing"])
perf_all = np.vstack((perf_q, perf_rebal, perf_nothing))
# drop the reward (last) colum
perf_all = perf_all[:, 0:3]

In [5]:
pd.DataFrame(perf_all, index=model_names,
             columns=["sharpe_ratio", "avg_return", "std_return"])             

,sharpe_ratio,avg_return,std_return
2days_pen_return_l=0,-0.006789,-274.230614,46229.252526
2days_pen_return_l=0_5,-0.006887,-274.230614,45472.290264
2days_sharpe_reward,-0.006789,-274.230614,46229.252526
30days_pen_return_l=0,-0.006214,-276.319854,59330.344216
30days_pen_return_l=0_5,-0.006146,-276.319854,64399.235279
30days_sharpe_reward,-0.005658,-276.319854,67036.591700
7days_pen_return_l=0,-0.005117,-274.606049,150175.598018
7days_pen_return_l=0_5,-0.006814,-274.606049,46253.487448
7days_sharpe_reward,-0.005103,-274.606049,150367.096073
rebalance,0.014386,816.995113,44559.715279


Cherry pick the plots to show, and make them pretty

In [6]:
plots_to_gen = [("2days_pen_return_l=0", "AVA_FCX"),
                ("2days_pen_return_l=0", "BMY_HES"),
                ("7days_sharpe_reward", "CPB_WDC"), 
                ("7days_sharpe_reward", "DGX_MS")]

In [7]:
def cust_plt(ax, data, label='', **kwargs):
    ax.plot_date(data.index, data, ls='solid', marker='', label=label, **kwargs)
    
lbl_str = '{:s} ({:.5f})'

In [8]:
for (mdl, port) in plots_to_gen:
    nothing = pd.read_csv(join(trd.MODEL_LOC, mdl, trd.NOTHING_DIR, port + ".csv"), index_col=0)
    rebal = pd.read_csv(join(trd.MODEL_LOC, mdl, trd.REBAL_DIR, port + ".csv"), index_col=0)
    q = pd.read_csv(join(trd.MODEL_LOC, mdl, trd.Q_PORT_DIR, port + ".csv"), index_col=0)

    f = plt.figure(figsize=(10,6))
    ax1 = f.gca()

    cust_plt(ax1, nothing.total, 'Do Nothing')
    cust_plt(ax1, rebal.total,'Rebalance 30')
    cust_plt(ax1, q.total, 'Q Learning')

    ax1.axhline(y=1E6, color='black')
    ax1.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
    ax1.set_ylim(bottom=0)
    plt.xticks(rotation=40)
    ax1.set_title('{:s}: {:s} & {:s}'.format(mdl, *port.split('_')))

    f.savefig('{:s}_{:s}'.format(mdl, port))
    plt.close(f)